In [1]:

from mmvae_hub.experiment_vis.utils import *
from mmvae_hub.utils.MongoDB import MongoDatabase

[16:08:26 CEST] Log file: /Users/Hendrik/Documents/master_4/MMVAE_Hub/mmvae_hub/logger/logs/UTC-20210610-140826.log


In [2]:
exp_db = MongoDatabase(training=False)
experiments = exp_db.connect()

In [3]:
print('There are currently ', experiments.count_documents({}), 'experiments in the database.')

There are currently  95 experiments in the database.


In [4]:
df = make_experiments_dataframe(experiments)
# df.style.set_properties(subset=['_id'], **{'width': '1px'})

'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'
'NoneType' object has no attribute 'items'


In [5]:
show_cols = ['score', 'amortized_flow','method', 'end_epoch', 'class_dim', 'beta','weighted_mixture','num_mods', '_id', 'score_lr', 'score_gen','score_prd', 'version', 'expvis_url']
df[show_cols].sort_values(by=['score'], ascending=False).head(10)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
35,6.324189,NaN,pfom_5,99,512.0,0.0,1.0,3.0,polymnist_pfom_2021_05_21_09_59_39_321199,5.258067,13.714500,1.641897,0.0.5-dev,https://ppb.hendrikklug.xyz/4ad1b0c934d2.pdf
40,5.715956,NaN,joint_elbo,619,512.0,2.5,0.0,3.0,polymnist_joint_elbo_2021_05_23_12_09_30_507370,5.471367,11.676500,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/a1da3920fa87.pdf
88,5.483672,0.0,joint_elbo,99,256.0,1.0,0.0,3.0,polymnist_joint_elbo_2021_06_09_17_38_18_207824,5.366400,11.084617,0.627032,0.0.9-dev,https://ppb.hendrikklug.xyz/03fdf6021a74.pdf
56,5.357850,1.0,planar_mixture_5,599,256.0,0.5,1.0,3.0,polymnist_planar_mixture_2021_05_27_21_22_46_7...,3.831200,12.242350,1.395396,0.0.5-dev,https://ppb.hendrikklug.xyz/52a8c8a4466d.pdf
39,5.251122,NaN,planar_mixture_5,469,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_23_12_10_02_9...,2.934167,12.819200,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/698265a99cef.pdf
0,5.122961,NaN,joint_elbo,299,64.0,2.5,NaN,3.0,polymnist_joint_elbo_2021_05_04_09_21_34_487039,5.252283,10.116600,0.000000,NaN,NaN
51,5.071122,NaN,planar_mixture_5,599,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_25_18_55_51_4...,2.858500,12.354867,1.198629,0.0.5-dev,https://ppb.hendrikklug.xyz/aa9810d2fb6d.pdf
61,4.937439,0.0,planar_mixture_5,599,256.0,0.5,0.0,3.0,polymnist_planar_mixture_2021_05_28_14_53_44_1...,2.918333,11.893983,1.469842,0.0.5-dev,https://ppb.hendrikklug.xyz/2abccea703f7.pdf
76,4.913078,0.0,pfom_5,599,512.0,1.0,0.0,3.0,polymnist_pfom_2021_06_01_22_06_59_642354,2.427533,12.311700,1.096989,0.0.7-dev,https://ppb.hendrikklug.xyz/95e4bf95f5c7.pdf
82,4.886306,0.0,pfom_5,1999,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_02_14_55_01_470328,2.274267,12.384650,2.153740,0.0.7-dev,https://ppb.hendrikklug.xyz/f1dd92c11883.pdf


# Method: Planar Mixture

In [6]:
df.loc[df['method'].str.startswith('planar_mixture')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
56,5.357850,1.0,planar_mixture_5,599,256.0,0.5,1.0,3.0,polymnist_planar_mixture_2021_05_27_21_22_46_7...,3.831200,12.242350,1.395396,0.0.5-dev,https://ppb.hendrikklug.xyz/52a8c8a4466d.pdf
39,5.251122,NaN,planar_mixture_5,469,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_23_12_10_02_9...,2.934167,12.819200,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/698265a99cef.pdf
51,5.071122,NaN,planar_mixture_5,599,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_25_18_55_51_4...,2.858500,12.354867,1.198629,0.0.5-dev,https://ppb.hendrikklug.xyz/aa9810d2fb6d.pdf
61,4.937439,0.0,planar_mixture_5,599,256.0,0.5,0.0,3.0,polymnist_planar_mixture_2021_05_28_14_53_44_1...,2.918333,11.893983,1.469842,0.0.5-dev,https://ppb.hendrikklug.xyz/2abccea703f7.pdf
59,4.808056,1.0,planar_mixture_5,549,256.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_27_21_23_18_0...,2.985600,11.438567,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/422165e67c1c.pdf
43,4.793644,NaN,planar_mixture_5,149,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_24_17_33_03_0...,2.795650,11.585283,0.000000,0.0.5-dev,NaN
44,4.774389,NaN,planar_mixture_5,449,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_25_08_07_00_3...,2.720467,11.602700,0.000000,0.0.5-dev,NaN
38,4.721506,NaN,planar_mixture_5,489,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_23_12_10_03_0...,2.807067,11.357450,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/57259190dc8c.pdf
54,4.653483,NaN,planar_mixture_5,309,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_25_11_37_30_8...,2.688217,11.272233,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/96d5e9ecc28a.pdf
47,4.557278,NaN,planar_mixture_0,98,512.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_25_20_25_27_6...,2.879633,10.792200,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/ecba70288b22.pdf


### More than 1 flow, 3 mods

In [7]:
df.loc[(df['method'].str.startswith('planar_mixture')) & (df['num_flows'])  & (df['num_mods'] > 1)][[*show_cols]].sort_values(by=['score'], ascending=False).head(3)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
56,5.357850,1.0,planar_mixture_5,599,256.0,0.5,1.0,3.0,polymnist_planar_mixture_2021_05_27_21_22_46_7...,3.831200,12.242350,1.395396,0.0.5-dev,https://ppb.hendrikklug.xyz/52a8c8a4466d.pdf
39,5.251122,NaN,planar_mixture_5,469,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_23_12_10_02_9...,2.934167,12.819200,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/698265a99cef.pdf
51,5.071122,NaN,planar_mixture_5,599,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_25_18_55_51_4...,2.858500,12.354867,1.198629,0.0.5-dev,https://ppb.hendrikklug.xyz/aa9810d2fb6d.pdf


### without flows
looks good without flows.

In [8]:
df.loc[(df['method'].str.startswith('planar_mixture')) & (df['num_flows'] == 0) ][[*show_cols]].sort_values(by=['score'], ascending=False).head(3)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
47,4.557278,NaN,planar_mixture_0,98,512.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_25_20_25_27_6...,2.879633,10.792200,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/ecba70288b22.pdf
45,4.304789,NaN,planar_mixture_0,98,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_25_09_30_13_8...,2.668200,10.246167,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/d4a404ff3418.pdf
46,4.272472,NaN,planar_mixture_0,98,512.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_25_17_39_12_8...,2.609267,10.208150,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/b3ae6e3c75ad.pdf


### 1 mod

In [9]:
df.loc[(df['method'].str.startswith('planar_mixture')) & (df['num_mods'] == 1) ][[*show_cols]].sort_values(by=['score'], ascending=False).head(3)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
14,0.736833,NaN,planar_mixture_0,299,512.0,2.5,NaN,1.0,polymnist_planar_mixture_2021_05_12_20_40_24_4...,0.113950,0.622883,0.289370,0.0.2-dev,https://ppb.hendrikklug.xyz/df669e71d32f.pdf
17,0.714333,NaN,planar_mixture_0,299,512.0,2.5,NaN,1.0,polymnist_planar_mixture_2021_05_14_15_57_55_7...,0.106133,0.608200,0.321916,0.0.2-dev,NaN
18,0.714083,NaN,planar_mixture_0,299,512.0,2.5,NaN,1.0,polymnist_planar_mixture_2021_05_14_21_10_44_2...,0.107267,0.606817,0.313426,0.0.3-dev,NaN


## Without amortized flows

In [10]:
df.loc[(df['method'].str.startswith('planar_mixture'))&(df['amortized_flow'] == 0.0)][[*show_cols]].sort_values(by=['score'], ascending=False).head(10)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
61,4.937439,0.0,planar_mixture_5,599,256.0,0.5,0.0,3.0,polymnist_planar_mixture_2021_05_28_14_53_44_1...,2.918333,11.893983,1.469842,0.0.5-dev,https://ppb.hendrikklug.xyz/2abccea703f7.pdf
53,4.537850,0.0,planar_mixture_5,99,256.0,1.0,1.0,3.0,polymnist_planar_mixture_2021_05_27_14_32_55_1...,3.104200,10.509350,0.458513,0.0.5-dev,https://ppb.hendrikklug.xyz/79f89ed3f3c7.pdf
65,4.414906,0.0,planar_mixture_5,599,256.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_28_14_54_01_0...,2.803883,10.440833,1.176529,0.0.5-dev,https://ppb.hendrikklug.xyz/7d211fdb5840.pdf
68,4.224222,0.0,planar_mixture_1,99,256.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_31_08_54_11_2...,2.546450,10.126217,0.967555,0.0.5-dev,https://ppb.hendrikklug.xyz/1f71d7551482.pdf
69,4.218622,0.0,planar_mixture_10,99,256.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_31_12_05_33_6...,2.610417,10.045450,0.987330,0.0.5-dev,https://ppb.hendrikklug.xyz/f4eb78c485f5.pdf
83,4.180678,0.0,planar_mixture_5,1999,256.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_06_02_14_51_16_6...,2.778600,9.763433,1.206549,0.0.7-dev,https://ppb.hendrikklug.xyz/3f77e986f5cc.pdf
70,4.161256,0.0,planar_mixture_20,99,256.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_31_15_45_48_1...,2.432133,10.051633,0.881162,0.0.6-dev,NaN
67,4.128467,0.0,planar_mixture_5,99,128.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_30_19_31_38_9...,2.491100,9.894300,0.752883,0.0.5-dev,https://ppb.hendrikklug.xyz/c450545ac62e.pdf
52,4.055856,0.0,planar_mixture_5,99,256.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_27_10_19_37_4...,2.375800,9.791767,0.971732,0.0.5-dev,https://ppb.hendrikklug.xyz/763904fafd25.pdf
66,4.022578,0.0,planar_mixture_5,99,64.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_05_30_16_01_45_1...,2.765000,9.302733,0.494348,0.0.5-dev,https://ppb.hendrikklug.xyz/3b56de32bea9.pdf


# Method: pfom


In [11]:
df.loc[df['method'].str.startswith('pfom')][[*show_cols]].sort_values(by=['score'], ascending=False)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
35,6.324189,NaN,pfom_5,99,512.0,0.0,1.0,3.0,polymnist_pfom_2021_05_21_09_59_39_321199,5.258067,13.714500,1.641897,0.0.5-dev,https://ppb.hendrikklug.xyz/4ad1b0c934d2.pdf
76,4.913078,0.0,pfom_5,599,512.0,1.0,0.0,3.0,polymnist_pfom_2021_06_01_22_06_59_642354,2.427533,12.311700,1.096989,0.0.7-dev,https://ppb.hendrikklug.xyz/95e4bf95f5c7.pdf
82,4.886306,0.0,pfom_5,1999,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_02_14_55_01_470328,2.274267,12.384650,2.153740,0.0.7-dev,https://ppb.hendrikklug.xyz/f1dd92c11883.pdf
74,4.725467,0.0,pfom_5,99,256.0,0.5,0.0,3.0,polymnist_pfom_2021_06_02_05_14_34_488367,2.856117,11.320283,1.024599,0.0.7-dev,https://ppb.hendrikklug.xyz/af7e76eaa8fd.pdf
62,4.723506,0.0,pfom_5,599,512.0,0.5,0.0,3.0,polymnist_pfom_2021_05_28_14_54_29_012526,1.840167,12.330350,1.050978,0.0.5-dev,https://ppb.hendrikklug.xyz/949b8297699f.pdf
90,4.481139,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_10_10_59_17_258140,3.701900,9.741517,0.627192,0.0.5-dev,https://ppb.hendrikklug.xyz/1420e90299eb.pdf
77,4.460817,0.0,pfom_0,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_02_12_03_22_476014,2.431450,10.951000,0.916068,0.0.7-dev,https://ppb.hendrikklug.xyz/4f23e2a615e1.pdf
57,4.350156,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_05_28_09_08_03_567569,3.694050,9.356417,0.401679,0.0.5-dev,https://ppb.hendrikklug.xyz/dd207fbd2e98.pdf
60,4.277417,0.0,pfom_5,99,256.0,1.0,1.0,3.0,polymnist_pfom_2021_05_28_14_44_39_476656,3.669750,9.162500,0.575645,0.0.5-dev,https://ppb.hendrikklug.xyz/68dafbd44b1f.pdf
64,4.049817,0.0,pfom_5,599,512.0,1.0,0.0,3.0,polymnist_pfom_2021_05_28_14_54_29_012764,1.242100,10.907350,0.526863,0.0.5-dev,https://ppb.hendrikklug.xyz/be3630d3e0d6.pdf


## Without amortized flows

In [12]:
df.loc[(df['method'].str.startswith('pfom'))&(df['amortized_flow'] == 0.0)][[*show_cols]].sort_values(by=['score'], ascending=False).head(10)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
76,4.913078,0.0,pfom_5,599,512.0,1.0,0.0,3.0,polymnist_pfom_2021_06_01_22_06_59_642354,2.427533,12.311700,1.096989,0.0.7-dev,https://ppb.hendrikklug.xyz/95e4bf95f5c7.pdf
82,4.886306,0.0,pfom_5,1999,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_02_14_55_01_470328,2.274267,12.384650,2.153740,0.0.7-dev,https://ppb.hendrikklug.xyz/f1dd92c11883.pdf
74,4.725467,0.0,pfom_5,99,256.0,0.5,0.0,3.0,polymnist_pfom_2021_06_02_05_14_34_488367,2.856117,11.320283,1.024599,0.0.7-dev,https://ppb.hendrikklug.xyz/af7e76eaa8fd.pdf
62,4.723506,0.0,pfom_5,599,512.0,0.5,0.0,3.0,polymnist_pfom_2021_05_28_14_54_29_012526,1.840167,12.330350,1.050978,0.0.5-dev,https://ppb.hendrikklug.xyz/949b8297699f.pdf
90,4.481139,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_10_10_59_17_258140,3.701900,9.741517,0.627192,0.0.5-dev,https://ppb.hendrikklug.xyz/1420e90299eb.pdf
77,4.460817,0.0,pfom_0,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_02_12_03_22_476014,2.431450,10.951000,0.916068,0.0.7-dev,https://ppb.hendrikklug.xyz/4f23e2a615e1.pdf
57,4.350156,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_05_28_09_08_03_567569,3.694050,9.356417,0.401679,0.0.5-dev,https://ppb.hendrikklug.xyz/dd207fbd2e98.pdf
60,4.277417,0.0,pfom_5,99,256.0,1.0,1.0,3.0,polymnist_pfom_2021_05_28_14_44_39_476656,3.669750,9.162500,0.575645,0.0.5-dev,https://ppb.hendrikklug.xyz/68dafbd44b1f.pdf
64,4.049817,0.0,pfom_5,599,512.0,1.0,0.0,3.0,polymnist_pfom_2021_05_28_14_54_29_012764,1.242100,10.907350,0.526863,0.0.5-dev,https://ppb.hendrikklug.xyz/be3630d3e0d6.pdf
72,3.908611,0.0,pfom_5,99,128.0,1.0,0.0,3.0,polymnist_pfom_2021_06_01_22_01_36_192602,2.313283,9.412550,0.531228,0.0.7-dev,https://ppb.hendrikklug.xyz/0ba7b499bd51.pdf


### without flows

In [13]:
df.loc[(df['method'].str.startswith('pfom')) & (df['num_flows'] == 0) ][[*show_cols]].sort_values(by=['score'], ascending=False).head(3)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
77,4.460817,0.0,pfom_0,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_02_12_03_22_476014,2.431450,10.951000,0.916068,0.0.7-dev,https://ppb.hendrikklug.xyz/4f23e2a615e1.pdf
31,2.992900,NaN,pfom_0,99,512.0,2.5,0.0,3.0,polymnist_pfom_2021_05_20_16_48_25_478210,1.766483,7.212217,0.187531,0.0.5-dev,https://ppb.hendrikklug.xyz/9ba46d23db0a.pdf


### base params

In [14]:
df.loc[(df['method'].str.startswith('pfom')) & (df['num_flows'] == 5) & (df['end_epoch'].astype(int) == 99)& (df['beta'] == 1) & (df['amortized_flow'] == 0) & (df['class_dim'] == 256) & (df['weighted_mixture'] == 0)& (df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(by=['score'], ascending=False)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
90,4.481139,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_10_10_59_17_258140,3.701900,9.741517,0.627192,0.0.5-dev,https://ppb.hendrikklug.xyz/1420e90299eb.pdf
57,4.350156,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_05_28_09_08_03_567569,3.694050,9.356417,0.401679,0.0.5-dev,https://ppb.hendrikklug.xyz/dd207fbd2e98.pdf
71,3.373983,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_01_16_13_13_192650,2.027633,8.094317,0.344550,0.0.7-dev,https://ppb.hendrikklug.xyz/2a754c1d71b7.pdf
86,1.903933,0.0,pfom_5,99,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_09_13_11_16_414931,1.141017,4.570783,0.191319,0.0.9-dev,https://ppb.hendrikklug.xyz/a7c575f3c886.pdf


# Runs with current version

In [15]:
version = BaseFlagsSetup.get_version_from_setup_config()
df.loc[df['version'] == version][[*show_cols]].sort_values(by=['score'], ascending=False)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url


# Runs with more than 1000 epochs

In [16]:
df.loc[df['end_epoch'].astype(int) > 1000][[*show_cols]].sort_values(by=['score'], ascending=False)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
82,4.886306,0.0,pfom_5,1999,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_02_14_55_01_470328,2.274267,12.384650,2.153740,0.0.7-dev,https://ppb.hendrikklug.xyz/f1dd92c11883.pdf
83,4.180678,0.0,planar_mixture_5,1999,256.0,1.0,0.0,3.0,polymnist_planar_mixture_2021_06_02_14_51_16_6...,2.778600,9.763433,1.206549,0.0.7-dev,https://ppb.hendrikklug.xyz/3f77e986f5cc.pdf
84,0.969822,0.0,pfom_5,1999,256.0,1.0,0.0,3.0,polymnist_pfom_2021_06_05_22_48_41_678669,0.704667,2.204800,0.010850,0.0.8-dev,https://ppb.hendrikklug.xyz/e94dd67d0ae8.pdf


# Method: joint_elbo

In [17]:
df.loc[df['method'].str.startswith('joint_elbo')][[*show_cols]].sort_values(by=['score'], ascending=False)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
40,5.715956,NaN,joint_elbo,619,512.0,2.5,0.0,3.0,polymnist_joint_elbo_2021_05_23_12_09_30_507370,5.471367,11.676500,0.000000,0.0.5-dev,https://ppb.hendrikklug.xyz/a1da3920fa87.pdf
88,5.483672,0.0,joint_elbo,99,256.0,1.0,0.0,3.0,polymnist_joint_elbo_2021_06_09_17_38_18_207824,5.366400,11.084617,0.627032,0.0.9-dev,https://ppb.hendrikklug.xyz/03fdf6021a74.pdf
0,5.122961,NaN,joint_elbo,299,64.0,2.5,NaN,3.0,polymnist_joint_elbo_2021_05_04_09_21_34_487039,5.252283,10.116600,0.000000,NaN,NaN
26,4.809444,NaN,joint_elbo,99,512.0,2.5,0.0,3.0,polymnist_joint_elbo_2021_05_19_10_59_36_964676,5.514483,8.913850,0.288514,0.0.4-dev,https://ppb.hendrikklug.xyz/2c1833766eda.pdf
4,3.913089,NaN,joint_elbo,299,64.0,0.5,NaN,3.0,polymnist_joint_elbo_2021_05_08_05_21_05_683057,3.932350,7.806917,0.611546,0.0.1-dev,NaN
2,3.322878,NaN,joint_elbo,299,32.0,0.5,NaN,3.0,polymnist_joint_elbo_2021_05_06_12_59_58_785343,3.239317,6.729317,0.312330,0.0.1-dev,NaN


# Method: poe

In [18]:
df.loc[df['method'].str.startswith('poe')][[*show_cols]].sort_values(by=['score'], ascending=False)

,score,amortized_flow,method,end_epoch,class_dim,beta,weighted_mixture,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url
85,0.956056,0.0,poe,99,256.0,1.0,0.0,3.0,polymnist_poe_2021_06_09_09_39_54_347753,0.704833,2.163333,0.007063,0.0.9-dev,https://ppb.hendrikklug.xyz/dc325835e094.pdf


# Generate samples with the model that has the highest score.

In [19]:
best_score_id = df[df.score == df.score.max()]._id.item()

print(best_score_id)






polymnist_pfom_2021_05_21_09_59_39_321199


In [20]:
show_generated_figs(_id = best_score_id)

TypeError: load_old_flags() got an unexpected keyword argument '_id'